In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression


In [6]:

Input_hourly = pd.read_csv('input_hourly.csv', parse_dates=['date'])

# split into input and output
X_site1 = Input_hourly.drop(["time_id", "date", "site1_utilization_percentage", "site2_utilization_percentage"], axis=1)
y_site1 = Input_hourly["site1_utilization_percentage"]

X_site2 = Input_hourly.drop(["time_id", "date", "site1_utilization_percentage", "site2_utilization_percentage"], axis=1)
y_site2 = Input_hourly["site2_utilization_percentage"]

# cross-validation
X_train_site1, X_test_site1, y_train_site1, y_test_site1 = train_test_split(X_site1, y_site1, test_size=0.3, random_state=42)

X_train_site2, X_test_site2, y_train_site2, y_test_site2 = train_test_split(X_site2, y_site2, test_size=0.3, random_state=42)

# normalize the data, save the original values and delete columns
X_train_site1_original = X_train_site1.copy()
X_test_site1_original = X_test_site1.copy()

X_train_site1 = X_train_site1.drop(["hour", "month", "dayOfTheWeek"], axis=1)
X_test_site1 = X_test_site1.drop(["hour", "month", "dayOfTheWeek"], axis=1)

scaler_site1 = StandardScaler()
scaler_site1.fit(X_train_site1)

X_train_site1 = scaler_site1.transform(X_train_site1)
X_test_site1 = scaler_site1.transform(X_test_site1)

X_train_site2_original = X_train_site2.copy()
X_test_site2_original = X_test_site2.copy()

X_train_site2 = X_train_site2.drop(["hour", "month", "dayOfTheWeek"], axis=1)
X_test_site2 = X_test_site2.drop(["hour", "month", "dayOfTheWeek"], axis=1)

scaler_site2 = StandardScaler()
scaler_site2.fit(X_train_site2)


StandardScaler()

In [8]:
def train_and_plot(X_train, y_train, X_test, y_test, site_name):
    # Train polynomial regression model
    degree = 3  # Adjust the degree as needed
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)

    # Predict on training set
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)

    # Predict on test set
    y_test_pred = model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_test_pred)

    plt.figure(figsize=(10, 6))
    plt.scatter(X_train["hour"], y_train, label='Actual (Train)')
    plt.scatter(X_test["hour"], y_test, label='Actual (Test)')

    # Sort the values for better visualization
    sort_order_train = np.argsort(X_train["hour"])
    sort_order_test = np.argsort(X_test["hour"])

    plt.plot(X_train["hour"].iloc[sort_order_train], y_train_pred[sort_order_train], label='Predicted (Train)', color='orange')
    plt.plot(X_test["hour"].iloc[sort_order_test], y_test_pred[sort_order_test], label='Predicted (Test)', color='red')

    plt.title(f'Polynomial Regression for {site_name} Utilization')
    plt.xlabel('Hour')
    plt.ylabel('Utilization Percentage')
    plt.legend()

    print(f"Mean Squared Error (Train): {mse_train}")
    print(f"Mean Squared Error (Test): {mse_test}")

# Train and plot for site1
train_and_plot(X_train_site1, y_train_site1, X_test_site1, y_test_site1, 'Site 1')

# Train and plot for site2
train_and_plot(X_train_site2, y_train_site2, X_test_site2, y_test_site2, 'Site 2')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

<Figure size 1000x600 with 0 Axes>